In [22]:
import pandas as pd
import cleaningtools as ct 
import file_config as fconfig
import merge_config as config
import assay_sample_config as aconfig
import numpy as np
import math


In [23]:
data=pd.read_csv('/Volumes/GoogleDrive/Shared drives/AMC Projects/_AZ_Kay/_Master Databases/drill assay samples master.csv')

In [24]:
aconfig.hole_id_formats[0]

'([a-zA-Z]{2}\\-{1}\\d{2}\\-{1}\\d{2})'

In [34]:
for col in data.columns:
    try:
        print(data[col].str.extract(aconfig.hole_id_formats[1]).squeeze().unique())
    except Exception as e: 
        print(e)

[nan]
[nan]
[nan]
[nan 'KM-20-14A' 'KM-20-10C' 'KM-20-10B' 'KM-20-10A' 'KM-20-03A'
 'KM-22-71A' 'KM-22-63D' 'KM-22-62C' 'KM-22-62B' 'KM-22-59A' 'KM-22-57C']
Can only use .str accessor with string values!
Can only use .str accessor with string values!
Can only use .str accessor with string values!
[nan]
[nan]
Can only use .str accessor with string values!
[nan]
Can only use .str accessor with string values!
[nan]
[nan 'KM-20-14A' 'KM-20-10C' 'KM-20-10B' 'KM-20-10A' 'KM-20-03A'
 'KM-22-71A' 'KM-22-63D' 'KM-22-62C' 'KM-22-62B' 'KM-22-59A' 'KM-22-57C']
[nan 'KM-22-71A' 'KM-22-63D' 'KM-22-62C' 'KM-22-62B' 'KM-22-59A'
 'KM-22-57C']
['KM-21-19K']
Can only use .str accessor with string values!
Can only use .str accessor with string values!


In [ ]:
def clean_for_merge(data):
    for col in data.columns:
        if 'depth_ft'in col.lower():
            data['from_ft']=data[col]
            data['to_ft']=data['from_ft'].shift(-1)-1
            #convert ft to meters
            data['from_m']=data['from_ft']* .3281
            data['to_m']=data['to_ft'] * .3281
            data=data.drop(data.filter(like='Depth').columns,axis=1)
        if ('hole' in col.lower())& ('id' in col.lower()):
            data[col]=data[col].astype(object)
            data.rename(columns={f'{col}':f'hole_id'},inplace=True)
        try: 
            data[col]=data[col].str.strip()
        except Exception as e:
            pass
            #print(f'{col}: {e}')
        data.rename(columns={f'{col}':f'{col.strip().lower()}'},inplace=True)
        data=data.drop(data.filter(like='unnamed'),axis=1)
        if 'from_ft'in col.lower():
            try:
                drop_index=data[data['from_ft'].isna()].index
                data=data.drop(drop_index,axis=0)
            except Exception as e:
                pass
                #print(f'{e}')
    if 'hole_id' not in data.columns:
        data['hole_id']=np.nan
    if 'recvd wt.'  in data.columns:
        try:
            data['recvd wt.']=pd.to_numeric(data['recvd wt.'],errors='coerce')
            data['from_ft']=pd.to_numeric(data['from_ft'],errors='coerce')
        except Exception as e:
            print(e)
            pass
    names=data.loc[:,data.columns.duplicated()].columns
    if len(names)>0:
        print(f'duplcated column: {names} drop or else the nasty merge bug')
        data=data.loc[:,~data.columns.duplicated()].copy()

    return data

In [ ]:
def merge_concat_onid(big_df,data,merge_col=['geo','sample_id','hole_id','from_ft','to_ft']):
    
    num_common=big_df[big_df.sample_id.isin(data.sample_id)].shape[0]
    print(f'{num_common} common ids, concat')

    
    if num_common==0:
        print(f'{num_common} common ids, concat')
        big_df=pd.concat([big_df,data],axis=0)
    else: 
        print (f'{num_common} common ids, merge')
        big_df=big_df.merge(data,on=merge_col,how='left')
    n_col=big_df.shape[1]
    print('duplicate columns: ', big_df.columns.duplicated().sum())
    print('duplicate indexs: ', big_df.index.duplicated().sum())
    print(f'{n_col} columns')
    return big_df.reset_index(drop=True)

In [ ]:
big_df=pd.DataFrame()
data_list=[]
prev_cols=[]
for file in config.merge_samples:
    print(f"####### merge: {file.split('/')[-1]} #######")
    data=pd.read_csv(file)
    data=clean_for_merge(data)
    data_list.append(data)
    data=data.astype(object)
    big_df=big_df.astype(object)
    big_cols=list(big_df.columns)
    small_cols=list(data.columns)
    
    dups=sorted([col for col in big_cols if col in small_cols])
    
    try:
        [dups.remove(x) for x in ['file','folder','start_depth','end_depth']if x in data.columns]
    except:
        pass
    print(f'common columns {dups}')


    
    big_index=list(big_df.index)
    small_index=list(data.index)
    
    same_ids= [id for id in big_index if id in small_index]
    n_ids=len(same_ids)
    if n_ids==0:
        print(f'{n_ids} common ids concat')
        big_df=pd.concat([big_df,data],axis=0,join='outer')
    else:
        print(f'{n_ids} common ids merge')
        big_df=big_df.merge(data,on=dups,how='outer')
    print(f'data shape {big_df.shape}')
cols=config.main_columns
not_dups=[i for i in big_df.columns if i not in cols]
big_df=pd.concat([big_df[cols],big_df[not_dups]],axis=1).set_index('sample_id')
big_df.sort_index(inplace=True)
#big_df=big_df.groupby(['sample_id']).apply(lambda x: x.fillna(method='ffill').fillna(method='bfill').drop_duplicates(keep='first')).droplevel(level=0)

In [ ]:
big_df

In [ ]:
big_df.to_excel('all_sampleid_test.xlsx')
big_df.to_excel('/Volumes/GoogleDrive/Shared drives/AMC Projects/_AZ_Kay/_Master Databases/all_sampleid_test.xlsx')



In [ ]:
big_df=pd.read_excel('all_sampleid_test.xlsx')
big_df[['from_ft','to_ft']]=big_df[['from_ft','to_ft']].fillna(0)
big_df

In [ ]:
lith=pd.read_csv(fconfig.lith_file)
lith=clean_for_merge(lith)
lith

In [ ]:
merge_cols=['sample_id','hole_id','from_ft','to_ft']
other_cols=[col for col in big_df.columns if col not in merge_cols]
zipped=zip(big_df['sample_id'],big_df['hole_id'], big_df['from_ft'], big_df['to_ft'])
depths=pd.DataFrame([(s_id,h_id, y) for s_id,h_id, start, end in zipped for y in np.arange(start, end,.5)],
                   columns=['sample_id','hole_id','depth'])
depths

In [ ]:
depths=depths.merge(big_df,on='sample_id',validate='1:m')

In [ ]:
merged=depths.merge(lith,left_on=['hole_id','depth'],right_on=['hole_id','from_ft'],how='outer')
other_cols=[col for col in depths if col not in merge_cols+'depths']
merged=pd.concat([merged[merge_cols+'depth'],merged[other_cols]],axis=1)


In [ ]:
merged.to_excel('merged_test.xlsx')